<a href="https://colab.research.google.com/github/adaley222/motor-classification/blob/main/Motor_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# A Convolutional Neural Network to Classify EEG Motor Imagery
## Author: Andrew Daley

This notebook is based on the research done by Lun et. al. found in this paper: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7522466/, as well as many others.

The purpose of the notebook is to use the Physionet MI-EEG dataset to train a convolutional neural network to classify motor-imagery events.

What that means simply, is that we have a dataset where subjects were instructed to imagine themselves closing their right or left fist, but not actually closing them. The experimenters used EEG machines to pick up the brain-wave activity of those events.

We can then use that EEG data to train a network to classify similar events in real-time. While this may seem trivial, the ability for a computer to recognize brain-wave events has wide-ranging uses for scenarios where someone is unable to interface with a phone or computer using their hands.

This is a pet project. I think that Brain-Computer Interfaces are fascinating, and while the idea of a chip in your brain is little more than a meme today, I believe some version of a BCI will become ubiquitious within our lifetimes.

Humor me as I explore the world of BCI, and learn a little about creating and training CNNs along the way. This is a work in progress, so if you come across this and see an error message below, please feel free to fix it for me =)

First, we'll install the packages and libraries we need.


In [ ]:
!pip install pyEDFlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 10.7 MB/s eta 0:00:00


In [ ]:
!pip install mne

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 16.5 MB/s eta 0:00:00


In [ ]:
import os
from urllib.error import HTTPError
import numpy as np
import pyedflib
import matplotlib.pyplot as plt
import pandas as pd
import scipy
from sklearn import svm, metrics
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import tensorflow as tf

from mne import Epochs, pick_types, events_from_annotations
from mne.channels import make_standard_montage
from mne.io import concatenate_raws, read_raw_edf
from mne.datasets import eegbci
from mne.decoding import CSP



from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, Dropout, BatchNormalization, Activation, MaxPooling2D, Flatten


My first iteration used a library called pyEDFlib to load the data. I was able to sucessfully write functions to process the data and train the network using that library.

However, after further research and learning from BCI professionals, I learned that a library called MNE is much more popular and provides much more powerful functionality for EEG pre-processing.

I am in the process of re-writing everything using MNE. However, we will begin with the code I wrote for PyEDFlib. Unfortunately, PyEDFlib cannot read the edf files in their standard format. There is a header value in the files that causes the library to fail to load the file. I have gone and changed the files manually, locally on my machine, and have been uploading them to colab every session.

This is an obvious problem with PyEDFlib, and one of the many reasons I am switching to MNE. However, walking through this pipeline gives us a good idea of how we need to engineer our data.

We start by creating a "master". EDF signals and annotations (event labels) are separate. We need to combine the signal with the appropriate annoations into a single object.

In [ ]:
def create_master(edf, signal_length, time_length):

    """
    Takes in an edf file and returns the dataset, as well as a master set

    Assumes file is a standard 64 signal edf

    removes last half second of file (which is 0 for my dataset)

    This is designed for two-minute recordings. The function will currently not work properly if time_length is not set to 120. This is a TODO.

    INPUT
    edf: an edf file read in with pyedflib.EdfReader()
    signal_length: length of signal (how many datapoints in each signal)
    time_length: length of experiment in seconds

    RETURNS
    dataset: ndarray (signal_length x #_signals), represents the signal for each node
    master_df: pandas dataframe (signal_length x (2 + #_signals)), with timestamp and annotation as first two columns

    """

    annotations = edf.readAnnotations()

    # removes last 0.5 seconds from record (trailing 0s)
    trunc_time = time_length - 0.5
    trunc_length = int(signal_length - signal_length * (1 - trunc_time/time_length))

    # creates a vector of time values by dividing up our time_length into signal_length parts
    timeArray = np.array([round(x,5) for x in np.arange(0,trunc_time,time_length/signal_length)])
    timeArray = timeArray.reshape(trunc_length,1)

    # creates codeArray, which is a vector of length signal_length (same as timeArray)
    # codeArray assigns the appropriate annotation at each point in timeArray
    intervals = np.append(annotations[0],[124.5])
    codes = annotations[2]
    codeArray = []
    counter = 1
    for timeVal in timeArray:
        if timeVal == 124.5:
            break
        elif timeVal / intervals[counter] == 1.0:
            counter += 1

        codeArray.append(codes[counter - 1])

    invertCodeArray = np.array(codeArray).reshape(trunc_length,1)
    numSignals = edf.signals_in_file
    signal_labels = edf.getSignalLabels()
    dataset = np.zeros((numSignals, edf.getNSamples()[0]))

    # creates array of (signal_source, signal) i.e. (node 64, signal of that node)
    # Then removes trailing 0 values
    for signal in np.arange(numSignals):
        dataset[signal, :] = edf.readSignal(signal)

    num_trailing_0s = signal_length - trunc_length
    dataset = dataset[:,:-num_trailing_0s].transpose()

    # combine arrays into master_df
    masterSet = np.concatenate((timeArray,invertCodeArray,dataset),axis=1)
    master_df = pd.DataFrame(masterSet, columns = [
                                         "timestamp",
                                         "label",
                                         "c1","c2","c3","c4","c5","c6","c7","c8","c9","c10","c11","c12","c13","c14","c15","c16","c17","c18","c19","c20","c21","c22","c23","c24","c25","c26","c27","c28","c29","c30","c31","c32","c33","c34","c35","c36","c37","c38","c39","c40","c41","c42","c43","c44","c45","c46","c47","c48","c49","c50","c51","c52","c53","c54","c56","c57","c58","c59","c60","c61","c62","c63","c64","c65"
                              ])

    return dataset, master_df

Now that we have the master, we need to pull random samples from the master. This is where I deviate from the research papers into what I believe is a more valuable model.

Real-time EEG data will be coming in rapidly, and will need to be sampled every fraction of a second to provide real-time output. Most research has used epoch style samples for training, which are not really how a BCI product would work in the real world.

Therefore, I randomly sample the data to create a new dataset of random, quarter-second bits that will represent an actual quarter-second sample taken in real-time. These samples are then combined into the new training dataset.

In [ ]:
def create_samples(eeg, sample_size, num_samples):

  """
  This function takes in an eeg dataframe and returns a designated number of samples of length sample_size and their labels
  These samples are random, but retain their time series order

  INPUT
  eeg: a pandas dataframe with label and channels as columns and time series indicies as rows
  sample_size: length of samples to return
  num_samples: number of samples to return

  OUTPUT
  samples: 2d numpy array with each  sample as a row
    dimensions are (num_samples, number of channels)

  labels: 1d numpy array with label for each sample
    length(num_samples)

  """

  if sample_size > eeg.shape[0]:
    print("sample_size must be less than length of eeg")
    return

  samples = np.zeros((num_samples, sample_size, eeg.shape[1]-2))
  labels = np.zeros(num_samples)

  for i in range(num_samples):
    start = np.random.randint(0, eeg.shape[0]-sample_size+1)
    df_sample = eeg[start:start+sample_size]
    sample = df_sample.drop(columns= ["timestamp", "label"]).to_numpy()

    # we're currently using the max value as the label. We may want to return and try other methods (averages, etc.)
    label = df_sample["label"].value_counts().idxmax()
    if label == "T0":
      label = 0
    elif label == "T1":
      label = 1
    elif label == "T2":
      label = 2

    samples[i] = sample
    labels[i] = label

  return samples, labels

In [ ]:
# Create eeg dataframe from edf file
dataset, master_df = create_master(S001R03, 20000, 125)

sample_size = 160
num_samples = 120

# Sample each dataframe randomly in one second intervals
samples, labels = create_samples(master_df, sample_size= sample_size, num_samples=num_samples)

Finally, I write a function to process an entire directory using the functions above, and create a final training set that is sampled from all edf files in a designated directory.

In [ ]:
def process_dir(dir_name, num_samples, sample_size):

  """
  Takes a directory of edf files, samples each using the create_samples function, and combines all samples into a master sample

  INPUT:
  dir_name: string, name of directory. Must be in cwd

  OUTPUT:
  master_sample: a numpy array of size (num_samples * train_dir_len, sample_size * train_dir_len, 64 * train_dir_len)
  """

  train_dir_len = len([name for name in os.listdir(dir_name)])

  master_sample = []
  master_labels = []

  for i, filename in enumerate(os.listdir(dir_name)):

    edf = pyedflib.EdfReader(dir_name + "/" + filename)

    dataset, master_df = create_master(edf, 20000, 125)

    samples, labels = create_samples(master_df, sample_size = sample_size, num_samples = num_samples)

    master_sample.append(samples)
    master_labels.append(labels)

  master_sample = np.array(master_sample)
  master_sample = master_sample.reshape((train_dir_len * num_samples, sample_size, 64))

  master_labels = np.array(master_labels)
  master_labels = master_labels.reshape((train_dir_len * num_samples))

  return master_sample, master_labels



In [ ]:
master_sample, master_labels = process_dir("",160,120)

NameError: name 'process_dir' is not defined

In [ ]:
master_sample = master_sample.reshape((640,120,64,1))

x_train,x_test,y_train,y_test = train_test_split(master_sample, master_labels,
                                               train_size = 0.8,
                                               test_size = 0.2,
                                               random_state = 24)

Now that we have our final training dataset, we can begin building our CNN. Again, the network is based on the research paper cited above. The details of the network can be found in the docstring below.

In [ ]:
# One hot encode labels
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

# Convert to tf tensors
x_train_tensor = tf.convert_to_tensor(x_train)
y_train_tensor = tf.convert_to_tensor(y_train)
x_test_tensor = tf.convert_to_tensor(x_test)
y_test_tensor = tf.convert_to_tensor(y_test)

In [ ]:
"""
Here we  build our network. We'll use the following architecture:

Dropout Rate: 50%
Learning Rate: 1 × 10^−5

L1 - Input Layer - num_files*sample size, sample_size, 64
L2 - Convolution1
    Activation
    Dropout
L3 - Convolution2
    Batch normalization
    Activation
L4 - Max Pooling1
L5 - Convolution3
    Activation
    Dropout
L6 - Max Pooling2
L7 - Convolution4
    Batch normalization
    Activation
L8 - Max Pooling3
L9 - Convolution5
    Batch normalization
    Activation
L10 - Max Pooling4
L11 - Flatten
      Fully Connected

We use catagorical cross-entropy for our loss function, and ADAM for our optimizer
"""

model = Sequential()

model.add(Conv2D(64, kernel_size = 3, activation= "relu", input_shape = (master_sample.shape[1], master_sample.shape[2], 1), padding="valid"))
model.add(Dropout(0.5))
model.add(Conv2D(32, kernel_size = 3, padding="valid"))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(MaxPooling2D())
model.add(Conv2D(32, kernel_size = 3, activation = "relu", padding="valid"))
model.add(Dropout(0.5))
model.add(MaxPooling2D())
model.add(Conv2D(32, kernel_size = 3, padding="valid"))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(Dropout(0.5))
model.add(MaxPooling2D())
model.add(Conv2D(32, kernel_size = 3, padding="valid"))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(MaxPooling2D())
model.add(Flatten())
model.add(Dense(3))



model.summary()
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_14 (Conv2D)           (None, 118, 62, 64)       640       
_________________________________________________________________
dropout_8 (Dropout)          (None, 118, 62, 64)       0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 116, 60, 32)       18464     
_________________________________________________________________
batch_normalization_8 (Batch (None, 116, 60, 32)       128       
_________________________________________________________________
activation_8 (Activation)    (None, 116, 60, 32)       0         
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 58, 30, 32)        0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 56, 28, 32)       

In [ ]:
model.fit(x_train_tensor, y_train_tensor, validation_data=(x_test_tensor, y_test_tensor), epochs=3)


Epoch 1/3
16/16 [==============================] - 24s 1s/step - loss: 6.1511 - accuracy: 0.3136 - val_loss: 3.8202 - val_accuracy: 0.2344
Epoch 2/3
16/16 [==============================] - 23s 1s/step - loss: 3.0714 - accuracy: 0.3903 - val_loss: 8.1526 - val_accuracy: 0.2734
Epoch 3/3
16/16 [==============================] - 22s 1s/step - loss: 1.4581 - accuracy: 0.4523 - val_loss: 6.0554 - val_accuracy: 0.4688


Now that we've outlined our basic data flow, I begin my second interation below using MNE

The code below downloads the data from the Physionet API using the MNE library. The function loads the files into a root directory, and the cell below moves them into a training directory.

You will need to type "y" when running the function to confirm you want to download the files into the default path. It's easier to just use the default and move the files in the next cell then to define the preferred folder for each file download.

Adjust the "subjects" and "runs" variables below to determine which subject/run combinations you'd like to download. Details are in the function docstring.

In [ ]:
subjects = [1,2,3,4,5]
runs = [3,4,7,8,11,12]

def download_physionet(subjects, runs):

  """
  Takes an int or list of subjects, and downloads that subject's respective runs from the physionet EEG motor imagery dataset

  PARAMETERS:
  ---------------
  subjects: int or list of ints to define which subjects to download. Values can range from 1-109 (inclusive)
  runs: int or list of ints to define which runs to download for each subject. Values can range from 1-14 (inclusive)
    run 1: Baseline (eyes open)
    run 2: Baseline (eyes closed)
    runs 3,7,11: Motor execution: left vs right hand
    runs 4,8,12: Motor imagery: left vs right hand
    runs 5,9,13: Motor execution: hands vs feet
    runs 6,10,14: Motor imagery: hands vs feet

  RETURNS:
  ---------------
    MNE raw object with all files concatonated
  """

  raw_fnames = []

  if type(subjects) == int:

    try:
      raw_fnames += eegbci.load_data(subjects, runs)

    except HTTPError as err:
      if err.code == 404:
        print("Subject values must be 1-109 and run values must be 1-14")
        return
      else:
        raise

  elif type(subjects) == list:

    for subject in subjects:
      try:
        raw_fnames += eegbci.load_data(subject, runs)

      except HTTPError as err:
        if err.code == 404:
          print("Subject values must be 1-109 and run values must be 1-14")
          return
        else:
          raise

  else:
    print("Subjects parameter must be int or list")


  raws = [read_raw_edf(f, preload=True) for f in raw_fnames]
  raw = concatenate_raws(raws)

  return

download_physionet(subjects, runs)

Download complete in 17s (14.9 MB)


Download complete in 17s (14.6 MB)


Download complete in 17s (14.9 MB)


Download complete in 16s (14.6 MB)


Download complete in 18s (14.6 MB)
Extracting EDF parameters from /root/mne_data/MNE-eegbci-data/files/eegmmidb/1.0.0/S001/S001R03.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...
Extracting EDF parameters from /root/mne_data/MNE-eegbci-data/files/eegmmidb/1.0.0/S001/S001R04.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...
Extracting EDF parameters from /root/mne_data/MNE-eegbci-data/files/eegmmidb/1.0.0/S001/S001R07.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...
Extracting EDF parameters from /root/mne_data/MNE-eegbci-data/files/eegmmidb/1.0.0/S001/S001R08.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs.

Moves files to training directory

In [ ]:
os.mkdir("/content/train")

for i, foldername in enumerate(os.listdir("/root/mne_data/MNE-eegbci-data/files/eegmmidb/1.0.0")):
  for j, filename in enumerate(os.listdir("/root/mne_data/MNE-eegbci-data/files/eegmmidb/1.0.0/"+ foldername)):
    os.system("mv /root/mne_data/MNE-eegbci-data/files/eegmmidb/1.0.0/" + foldername + "/" + filename + " /content/train")



Next we need to combine the signal and annotations into a master like we did before using PyEDFlib. This is currently my TODO, so the code below is incomplete, and largely a set of experimental code blocks.

However, if you continue scrolling you can find some basic ML models I created out of curiousity, as well as some very basic exploratory analysis.

In [ ]:
eegbci.standardize(raw)
montage = make_standard_montage('standard_1005')
raw.set_montage(montage)

raw.rename_channels(lambda x: x.strip('.'))

events, _ = events_from_annotations(raw, event_id=dict(T1=2, T2=3))

NameError: ignored

In [ ]:
data = raw.get_data()
type(data)
data.shape

(64, 594240)

In [ ]:
len(events)

450

In [ ]:
test_mne = read_raw_edf("/content/train/S001R03.edf", preload=True)
test_mne.get_data().shape
data = test_mne.get_data()

Extracting EDF parameters from /content/train/S001R03.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...


In [ ]:
test = pyedflib.EdfReader("/content/sample_data/S001R03.edf")

In [ ]:
# n = test.signals_in_file
# signal_labels = test.getSignalLabels()
# sigbufs = np.zeros((n, test.getNSamples()[0]))
# for i in np.arange(n):
#     sigbufs[i, :] = test.readSignal(i)

In [ ]:
numSignals = test.signals_in_file

dataset = np.zeros((numSignals, test.getNSamples()[0]))

# creates array of (signal_source, signal) i.e. (node 64, signal of that node)
# Then removes trailing 0 values
for signal in np.arange(numSignals):
    dataset[signal, :] = test.readSignal(signal)

In [ ]:
dataset.shape == data.shape

True

In [ ]:
annotations = test.readAnnotations()
annotations

(array([  0. ,   4.2,   8.3,  12.5,  16.6,  20.8,  24.9,  29.1,  33.2,
         37.4,  41.5,  45.7,  49.8,  54. ,  58.1,  62.3,  66.4,  70.6,
         74.7,  78.9,  83. ,  87.2,  91.3,  95.5,  99.6, 103.8, 107.9,
        112.1, 116.2, 120.4]),
 array([4.2, 4.1, 4.2, 4.1, 4.2, 4.1, 4.2, 4.1, 4.2, 4.1, 4.2, 4.1, 4.2,
        4.1, 4.2, 4.1, 4.2, 4.1, 4.2, 4.1, 4.2, 4.1, 4.2, 4.1, 4.2, 4.1,
        4.2, 4.1, 4.2, 4.1]),
 array(['T0', 'T2', 'T0', 'T1', 'T0', 'T1', 'T0', 'T2', 'T0', 'T2', 'T0',
        'T1', 'T0', 'T1', 'T0', 'T2', 'T0', 'T1', 'T0', 'T2', 'T0', 'T2',
        'T0', 'T1', 'T0', 'T1', 'T0', 'T2', 'T0', 'T1'], dtype='<U2'))

[link text](https://)

In [ ]:
def create_master(edf, signal_length, time_length):

   """
    Takes in an edf file and returns the dataset, as well as a master set

    Assumes file is a standard 64 signal edf

    removes last half second of file (which is 0 for my dataset)

    This is designed for two-minute recordings. The function will currently not work properly if time_length is not set to 120. This is a TODO.

    INPUT
    edf: an edf file read in with pyedflib.EdfReader()
    signal_length: length of signal (how many datapoints in each signal)
    time_length: length of experiment in seconds

    RETURNS
    dataset: ndarray (signal_length x #_signals), represents the signal for each node
    master_df: pandas dataframe (signal_length x (2 + #_signals)), with timestamp and annotation as first two columns

    """



In [ ]:
samples.shape()

(120, 160, 64)

In [ ]:
master_sample.shape

(640, 120, 64)

Here, we do some traditional machine learning algorithms (SVM and Logistic regression to start) and check their accuracy and speed to compare to our CNN.

However, these don't use the sampled data as discussed, and were really more for my own curiosity/exploration.

In [ ]:
traditional_labels = master_df["label"]

x_train,x_test,y_train,y_test=train_test_split(dataset, traditional_labels,
                                               train_size = 0.8,
                                               test_size = 0.2,
                                               random_state = 24)

In [ ]:
svm_classifier = svm.SVC()
svm_classifier.fit(x_train, y_train)

svm_pred = svm_classifier.predict(x_test)

print("Accuracy:", metrics.accuracy_score(y_test,svm_pred) )

In [ ]:
# takes a single row from the test set and times it
sample = x_test[0].reshape(1,-1)
%timeit svm_classifier.predict(sample)

In [ ]:
log_classifier = LogisticRegression()
log_classifier.fit(x_train,y_train)

log_pred = log_classifier.predict(x_test)

print(metrics.classification_report(y_test, log_pred))

In [ ]:
%timeit log_classifier.predict(sample)

Here we begin to build our CNN

This was some code I wrote while I considered using a 3d CNN. However, I found it would make real-time processing slow, was unnecessarily complicated, and decided against using it. I've kept the code in case I change my mind one day, or want to explore it further.

In [ ]:
"""
This was an attempt to reshape the signal into a 4d tensor using 3 of the
dimensions as input for the CNN, but I realized that doing that transformation
would make real time processing slow.

It is also incorrect as related to the research article I read. The re-shaping
is not as needed.

# Do this num_indicies times and stack
frame1 = dataset3[0:8,:]
frame2 = dataset3[1:9,:]
frame3 = dataset3[2:10,:]
frame4 = dataset3[3:11,:]
frame5 = dataset3[4:12,:]
frame6 = dataset3[5:13,:]

c = np.stack((frame1, frame2, frame3, frame4, frame5, frame6))
# c = c.reshape(8,64,6)
print(dataset3.shape[0])
"""

def create_3D(dataset, frame_size, chunk_size):

  """
  INPUT
  dataset: an EEG dataset of type array and shape (time_stamp x number of channels)
  chunk_size: number of frames pooled together to create a chunk i.e. depth of output tensor
  frame_size: number of timestamps pooled together to create a frame i.e. width of output tensor

  The height of the output tensor is the number of channels i.e. dataset.shape[1]

  OUTPUT

  tensor: A 4d tensor of shape (index, frame_size, num_channels, chunk_size).

  each index is the 3d tensor used as input for the CNN


  In my case, I'll use a frame size of 8 and a chunk size of 6
  """

  num_indices = dataset.size / dataset.shape[1] / frame_size / chunk_size

  if num_indices.is_integer():
    num_indices = int(num_indices)

  else:
    print("Proposed chunk and/or frame size do not fit data")
    return

  try:

    return np.reshape(dataset, (num_indices, frame_size, dataset.shape[1], chunk_size), order = "F")

  except ValueError:

    print("Proposed chunk and/or frame size do not fit data")

This was a brief analysis of the "moments" in the data, which are the events. I wanted to see if the events really did have higher mean values.

In [ ]:
moments = master_df[master_df["label"] == "T1"]
moments.shape

(5248, 66)

In [ ]:
moment_data = moments.iloc[:,2:]
moment_data

,c1,c2,c3,c4,c5,c6,c7,c8,c9,c10,c11,c12,c13,c14,c15,c16,c17,c18,c19,c20,c21,c22,c23,c24,c25,c26,c27,c28,c29,c30,c31,c32,c33,c34,c35,c36,c37,c38,c39,c40,c41,c42,c43,c44,c45,c46,c47,c48,c49,c50,c51,c52,c53,c54,c56,c57,c58,c59,c60,c61,c62,c63,c64,c65
2000,-11.0,10.0,16.0,22.0,28.0,21.0,32.0,8.0,5.0,9.0,14.0,87.0,23.0,18.0,-15.0,-1.0,5.0,6.0,20.0,29.0,27.0,-50.0,-8.0,-23.0,-60.0,-55.0,6.0,15.0,2.0,-45.0,-17.0,54.0,7.0,4.0,-9.0,17.0,35.0,17.0,-23.0,3.0,-26.0,-4.0,-33.0,-4.0,-40.0,13.0,-14.0,-8.0,-13.0,-3.0,7.0,8.0,16.0,8.0,4.0,-3.0,-10.0,-8.0,3.0,-16.0,-8.0,-21.0,-21.0,-51.0
2001,-31.0,-8.0,-5.0,-2.0,5.0,-3.0,7.0,-13.0,-12.0,-8.0,-7.0,69.0,-2.0,-7.0,-34.0,-17.0,-11.0,-12.0,1.0,8.0,5.0,-69.0,-31.0,-48.0,-83.0,-78.0,-22.0,-13.0,-15.0,-70.0,-46.0,21.0,-21.0,-27.0,-36.0,-6.0,10.0,-14.0,-50.0,-20.0,-54.0,-40.0,-33.0,-4.0,-52.0,-11.0,-30.0,-28.0,-25.0,-22.0,-8.0,-8.0,1.0,-5.0,-10.0,-20.0,-23.0,-19.0,-3.0,-9.0,-9.0,-16.0,-20.0,-48.0
2002,-49.0,-26.0,-21.0,-15.0,-6.0,-11.0,-5.0,-29.0,-28.0,-18.0,-11.0,68.0,-4.0,-12.0,-40.0,-26.0,-20.0,-15.0,-4.0,7.0,5.0,-64.0,-27.0,-46.0,-86.0,-80.0,-29.0,-15.0,-17.0,-78.0,-54.0,11.0,-34.0,-39.0,-45.0,-17.0,6.0,-18.0,-63.0,-19.0,-59.0,-18.0,-29.0,-11.0,-56.0,-12.0,-24.0,-33.0,-42.0,-37.0,-22.0,-16.0,-5.0,-8.0,-13.0,-25.0,-35.0,-34.0,-8.0,-15.0,-20.0,-22.0,-16.0,-43.0
2003,-78.0,-61.0,-53.0,-38.0,-28.0,-27.0,-11.0,-61.0,-62.0,-49.0,-33.0,44.0,-18.0,-15.0,-68.0,-59.0,-48.0,-42.0,-26.0,-10.0,-3.0,-90.0,-45.0,-51.0,-113.0,-106.0,-49.0,-30.0,-25.0,-108.0,-68.0,-11.0,-63.0,-66.0,-65.0,-41.0,-6.0,-17.0,-90.0,-14.0,-63.0,12.0,-56.0,-42.0,-85.0,-4.0,-58.0,-57.0,-61.0,-54.0,-38.0,-31.0,-20.0,-15.0,-15.0,-40.0,-50.0,-45.0,-16.0,-20.0,-33.0,-34.0,-10.0,-42.0
2004,-81.0,-67.0,-61.0,-44.0,-33.0,-25.0,-4.0,-50.0,-63.0,-51.0,-39.0,40.0,-18.0,-5.0,-68.0,-60.0,-46.0,-42.0,-23.0,-10.0,4.0,-79.0,-31.0,-45.0,-90.0,-87.0,-40.0,-21.0,-13.0,-95.0,-49.0,-3.0,-47.0,-51.0,-53.0,-23.0,10.0,-5.0,-68.0,1.0,-69.0,5.0,-64.0,-45.0,-84.0,-8.0,-54.0,-50.0,-48.0,-42.0,-26.0,-24.0,-14.0,-14.0,-14.0,-34.0,-44.0,-35.0,-10.0,-14.0,-37.0,-28.0,6.0,-23.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19915,23.0,16.0,14.0,10.0,10.0,2.0,17.0,23.0,24.0,24.0,30.0,21.0,4.0,-4.0,29.0,44.0,33.0,33.0,21.0,11.0,1.0,10.0,18.0,12.0,-12.0,-7.0,8.0,32.0,64.0,-3.0,-18.0,17.0,-1.0,-2.0,2.0,3.0,21.0,47.0,10.0,21.0,29.0,4.0,24.0,10.0,34.0,16.0,32.0,43.0,51.0,41.0,47.0,42.0,25.0,12.0,13.0,58.0,53.0,56.0,42.0,46.0,76.0,68.0,86.0,32.0
19916,23.0,23.0,24.0,20.0,23.0,14.0,22.0,31.0,31.0,38.0,47.0,44.0,21.0,4.0,34.0,54.0,46.0,51.0,46.0,41.0,27.0,11.0,20.0,8.0,4.0,8.0,8.0,27.0,50.0,19.0,-5.0,18.0,4.0,0.0,4.0,4.0,14.0,23.0,26.0,19.0,43.0,8.0,36.0,-14.0,51.0,66.0,53.0,54.0,66.0,54.0,69.0,72.0,55.0,48.0,50.0,61.0,52.0,72.0,72.0,79.0,72.0,82.0,124.0,51.0
19917,23.0,14.0,11.0,3.0,5.0,-3.0,1.0,21.0,19.0,25.0,33.0,37.0,13.0,2.0,23.0,36.0,38.0,41.0,41.0,36.0,27.0,-5.0,-17.0,-30.0,9.0,13.0,-15.0,-1.0,17.0,22.0,-13.0,3.0,-12.0,-20.0,-14.0,-14.0,-14.0,-8.0,23.0,4.0,45.0,29.0,30.0,27.0,29.0,52.0,18.0,26.0,49.0,39.0,62.0,69.0,55.0,46.0,44.0,24.0,24.0,66.0,74.0,85.0,62.0,97.0,156.0,75.0
19918,38.0,27.0,28.0,19.0,19.0,12.0,9.0,34.0,32.0,40.0,51.0,55.0,27.0,20.0,22.0,42.0,46.0,57.0,58.0,53.0,38.0,26.0,6.0,1.0,42.0,44.0,18.0,25.0,37.0,50.0,14.0,32.0,24.0,19.0,20.0,22.0,9.0,10.0,50.0,25.0,46.0,63.0,32.0,18.0,26.0,37.0,-1.0,18.0,45.0,38.0,65.0,65.0,49.0,33.0,25.0,5.0,11.0,53.0,51.0,43.0,44.0,81.0,140.0,66.0


In [ ]:
print(moment_data.astype(float).stack().mean())
print(moment_data.astype(float).stack().std())

-1.5217434022484757
69.9608073016054


In [ ]:
rest_data = master_df[master_df["label"] == "T0"].iloc[:,2:]
rest_data.shape

(10080, 64)

In [ ]:
print(rest_data.astype(float).stack().mean())
print(rest_data.astype(float).stack().std())

-6.537375992063492
62.031341693062885


Seems like the moments really do have higher means. I obviously haven't done any hypothesis tests or anything, but a quick glance shows that we're probably on to something.